In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\wine_quality_predication_project\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\wine_quality_predication_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Creating entity, entity is nothing but it is custom return type of the function.
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManger:
    def __init__(self,
                  config_filepath= CONFIG_FILE_PATH,
                  params_filepath=PARAMS_FILE_PATH,
                  schema_filepath=  SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.parms=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir, 
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [9]:
# Components.
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size


In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f"{filename} download! with follwing info: \n{header}")

        else: 
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        Zip_file_path : str
        Extreactes the zip file into the data directory
        Function return None.
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try: 
    config=ConfigurationManger()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config =data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-21 19:59:41,976 : INFO: common : Yaml file: config\config.yaml loaded sucessfully]
[2023-09-21 19:59:41,980 : INFO: common : Yaml file: params.yaml loaded sucessfully]
[2023-09-21 19:59:41,983 : INFO: common : Yaml file: schema.yaml loaded sucessfully]
[2023-09-21 19:59:41,987 : INFO: common : Created directory as : artifacts]
[2023-09-21 19:59:41,992 : INFO: common : Created directory as : artifacts/data_ingestion]


[2023-09-21 19:59:45,320 : INFO: 2039783704 : artifacts/data_ingestion/data.zip download! with follwing info: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1544:23B0E2:74333:A2564:650C4FD5
Accept-Ranges: bytes
Date: Thu, 21 Sep 2023 14:14:45 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830035-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1695305685.464724,VS0,VE345
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: ac9fb8bb82ba9ff0dafe9f64682acf0aad984742
Expires: Thu, 21 Sep 2023 14:19:45 GMT
Source-Age: 0

]
